# Demonstrate ``TuneAxis``

In this example, we demonstrate the `APS_BlueSky_tools.plans.TuneAxis()` plan.  The `TuneAxis()` support may be used to align (a.k.a. *tune*) a signal against an axis.

We'll use a software-only (not connected to hardware) motor as a positioner.  Here, we prepare a signal that is a computation based on the value of our positioner.  The computed signal is a model of a realistic diffraction peak one might encounter in a powder diffraction scan.  The model peak is a pseudo-voigt function to which some noise has been added.  Random numbers are used to modify the ideal pseudo-voigt function so as to simulate a realistic signal.

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *BestEffortCallback()* code.

In [1]:
from ophyd import EpicsMotor
from APS_BlueSky_tools.synApps_ophyd import swaitRecord, swait_setup_random_number
from APS_BlueSky_tools.examples import SynPseudoVoigt
from APS_BlueSky_tools.plans import TuneAxis
from bluesky.callbacks import LiveTable
import numpy as np

from bluesky import RunEngine
RE = RunEngine({})

Figure out which workstation we are running.  The *mint-vm* host has a different IOC prefix.

In [2]:
import socket
if socket.gethostname().find("mint-vm") >= 0:
    prefix = "prj:"
else:
    prefix = "xxx:"

Connect to our motor *before* we create the simulated detector signal.

In [3]:
m1 = EpicsMotor(prefix+"m1", name="m1")
m1.wait_for_connection()

Define a starting position, we'll use this later in the demo.

In [4]:
m1.move(-1.5)
starting_position = m1.position

## Setup the simulated detector signal.  

Randomize the values a bit so that we have something interesting to find with `TuneAxis()`.

In [5]:
spvoigt = SynPseudoVoigt(
    'spvoigt', m1, 'm1', 
    center=-1.5 + 0.4*np.random.uniform(), 
    eta=0.2 + 0.5*np.random.uniform(), 
    sigma=0.001 + 0.05*np.random.uniform(), 
    scale=1e5,
    bkg=0.01*np.random.uniform())

Reveal the actual values.  These are the answers we expect to discover.

In [6]:
print("spvoigt.scale: ", spvoigt.scale)
print("spvoigt.center: ", spvoigt.center)
print("spvoigt.sigma: ", spvoigt.sigma)
print("spvoigt.eta: ", spvoigt.eta)
print("spvoigt.bkg: ", spvoigt.bkg)

spvoigt.scale:  100000.0
spvoigt.center:  -1.20696530881405
spvoigt.sigma:  0.041924455226982904
spvoigt.eta:  0.2937992411916956
spvoigt.bkg:  0.007137869490883047


We will add the actual values as metadata to these scans.

In [7]:
md = dict(
    activity = "TuneAxis development and testing",
    peak_model = "pseudo Voigt",
    peak_scale = spvoigt.scale,
    peak_center = spvoigt.center,
    peak_sigma = spvoigt.sigma,
    peak_eta = spvoigt.eta,
    peak_bkg = spvoigt.bkg
    )

## Set up the tuner

Create a *TuneAxis()* object.  The *tuner* needs to know the positioner, what range to scan to find the peak, *and* it needs the name of the signal to be scanned (since the signal list may have more than one signal).

In [8]:
tuner = TuneAxis([spvoigt], m1, signal_name=spvoigt.name)
tuner.width = 2.5
tuner.step_factor = tuner.num/2.5

Configure the *LiveTable* to also show the simulated detector signal.

In [9]:
live_table = LiveTable(["m1", "spvoigt"])
#spvoigt.read_attrs = ["value"]

# Multi-pass tune

Execute multiple passes to refine the centroid determination.
Each subsequent pass will reduce the width of the next scan by ``step_factor``.

In [10]:
RE(tuner.multi_pass_tune(), live_table, md=md)

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 23:54:12.1 |   -2.75000 |    735.460 |
|         2 | 23:54:12.6 |   -2.47000 |    746.122 |
|         3 | 23:54:13.1 |   -2.19000 |    767.128 |
|         4 | 23:54:13.6 |   -1.92000 |    815.007 |
|         5 | 23:54:14.1 |   -1.64000 |    986.615 |
|         6 | 23:54:14.6 |   -1.36000 |   2855.119 |
|         7 | 23:54:15.1 |   -1.08000 |   4322.361 |
|         8 | 23:54:15.6 |   -0.81000 |   1037.875 |
|         9 | 23:54:16.1 |   -0.53000 |    826.038 |
|        10 | 23:54:16.6 |   -0.25000 |    770.068 |
+-----------+------------+------------+------------+
generator TuneAxis.multi_pass_tune ['99522b5a'] (scan num: 1)
x : m1
y : spvoigt
cen : -1.1707266098058466
com : -1.3523909322853631
fwhm : 0.4763131715445944
min : [  -2.75        735.45968109]
max : [ -1.08000000e+00   4.32236078e+03]
crossings : [-1.40888

('99522b5a-ab40-46a3-87c2-0fdac614d585',
 '0e21c496-7c4d-482d-b4a2-fd1497e302d9',
 'b0752679-0e95-4194-901d-75c77b4ea60b',
 '3872201b-4471-4b11-8fd4-40af3a576ac1')

Show the results from the multi-pass tuning.

In [11]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
for stat in tuner.stats:
    print("--", stat.cen, stat.fwhm)
print("m1=", m1.position, "", "det=", spvoigt.value)

final:  -1.2070932473
max (-1.21, 100375.88502190913)
min (-1.23, 84007.39214119091)
-- -1.17072660981 0.476313171545
-- -1.20488359243 0.0933151889449
-- -1.2070932473 0.0854482361361
-- -1.20693944579 0.0309435628144
m1= -1.21  det= 84007.3921412


Compare the final position (just printed) with the expected value shown a couple steps back.

## Single-pass tune

Repeat but with only one pass.  Reset the motor to the starting position and increase the number of steps by a factor of three.

In [12]:
m1.move(starting_position)
tuner.num *= 3
RE(tuner.tune(), live_table, md=md)

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 23:54:27.3 |   -2.75000 |    735.460 |
|         2 | 23:54:27.6 |   -2.66000 |    738.225 |
|         3 | 23:54:28.0 |   -2.58000 |    741.153 |
|         4 | 23:54:28.2 |   -2.49000 |    745.123 |
|         5 | 23:54:28.5 |   -2.41000 |    749.424 |
|         6 | 23:54:28.8 |   -2.32000 |    755.412 |
|         7 | 23:54:29.1 |   -2.23000 |    763.045 |
|         8 | 23:54:29.4 |   -2.15000 |    771.740 |
|         9 | 23:54:29.7 |   -2.06000 |    784.582 |
|        10 | 23:54:30.0 |   -1.97000 |    802.215 |
|        11 | 23:54:30.3 |   -1.89000 |    824.059 |
|        12 | 23:54:30.6 |   -1.80000 |    859.890 |
|        13 | 23:54:30.9 |   -1.72000 |    908.682 |
|        14 | 23:54:31.2 |   -1.63000 |    999.539 |
|        15 | 23:54:31.5 |   -1.54000 |   1172.117 |
|        16 | 23:54:31.8 |   -1.46000 |   1498

('9a38f942-aff8-4ae9-85e5-5807e392af6a',)

Compare the single-pass scan with the previous multi-pass scan.  Each used the same number of points overall.  

The results are comparable but we already knew the position of the peak approximately.

In [13]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
print("centroid", tuner.peaks.cen)
print("FWHM", tuner.peaks.fwhm)
print("m1=", m1.position, "", "det=", spvoigt.value)

final:  -1.2011256428
max (-1.2, 98956.670376535956)
min (-2.75, 735.45968108701959)
centroid -1.2011256428
FWHM 0.101858427768
m1= -1.2  det= 770.067766366
